# Data Preprocess

In [1]:
import pandas as pd

WORKDIR = '/content/drive/MyDrive/Data Mining/'

In [2]:
df_train = pd.read_csv(f'{WORKDIR}/Batch_answers - train_data (no-blank).csv' )

# drop unnecessary columns
df_train = df_train.drop(['Unnamed: 6', 'total no.: 7987'], axis=1)

# remove quotes
df_train['q'] = df_train['q'].str.strip('"')
df_train['r'] = df_train['r'].str.strip('"')
df_train["q'"] = df_train["q'"].str.strip('"')
df_train["r'"] = df_train["r'"].str.strip('"')

# drop duplicated rows
df_train = df_train.drop_duplicates()

# numerical s
df_train['s_label'] = (df_train['s'] == 'AGREE').astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [3]:
df_train['sentence'] = df_train['q'] + '[SEP]' + df_train['r']
df_train['id'] = df_train.index

# FIXME: drop length > 360
df_train = df_train[(df_train['sentence']).apply(lambda x: len(x)) < 360]

df_train[['id', 'sentence']].sample(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,sentence
19060,19060,God is not a racist .[SEP]But many christians ...
2959,2959,Do your own research . I am not interested in ...
21263,21263,Should the Georgia General Assembly go back to...


In [4]:
# mark start & end of q
match = []
start_pos = []
end_pos = []

for index, row in df_train.iterrows():
  sentence = row['sentence']
  sentence_words = sentence.split()
  answer = row['q\'']
  answer_words = answer.split()
  idx = 0
  start = -1
  success = False
  for i, w in enumerate(sentence_words):
    if w == answer_words[idx]:
      if idx == 0:
        start = i
      idx += 1
      if idx == len(answer_words):
        success = True
        break
  match.append(success)
  start_pos.append(start)
  end_pos.append(start + len(answer_words))

df_train['match_q'] = match
df_train['start_q'] = start_pos
df_train['end_q'] = end_pos

In [5]:
# mark start & end of r
match = []
start_pos = []
end_pos = []

for index, row in df_train.iterrows():
  sentence = row['sentence']
  sentence_words = sentence.split()
  answer = row['r\'']
  answer_words = answer.split()
  idx = 0
  start = -1
  success = False
  for i, w in enumerate(sentence_words):
    if w == answer_words[idx]:
      if idx == 0:
        start = i
      idx += 1
      if idx == len(answer_words):
        success = True
        break
  match.append(success)
  start_pos.append(start)
  end_pos.append(start + len(answer_words))

df_train['match_r'] = match
df_train['start_r'] = start_pos
df_train['end_r'] = end_pos

In [6]:
# to-fix
df_train.match_q.value_counts() / len(df_train)

True     0.761704
False    0.238296
Name: match_q, dtype: float64

In [7]:
# to-fix
df_train.match_r.value_counts() / len(df_train)

False    0.549992
True     0.450008
Name: match_r, dtype: float64

In [8]:
df_train = df_train[df_train['match_q']==True]
df_train = df_train[df_train['match_r']==True]
len(df_train)

4564

In [9]:
length = len(df_train)
train = df_train[:int(length*0.8)]
test = df_train[int(length*0.8):]

In [10]:
import csv
col = ["Sentence #", "Word", "POS", "Tag"]

with open(f'{WORKDIR}/train_ner.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(col)
    for index, row in train.iterrows():
      words = row['sentence'].split()
      for i in range(len(words)):
        data = []
        if i == 0:
          data.append(f'Sentence: {row["id"]}')
        else:
          data.append(None)
        data.append(words[i])
        data.append(None)
        data.append('O' if (row['start_q'] <= i < row['end_q'] or 
                            row['start_r'] <= i < row['end_r']) else 'X')
        writer.writerow(data)

# Bert
refer to [Custom Named Entity Recognition with BERT.ipynb](https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb)

In [11]:
!pip install transformers==3.1.0
!pip install seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [13]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [14]:
data = pd.read_csv(f"{WORKDIR}/train_ner.csv", encoding='unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 15,alright,NaN,X
1,NaN,how,NaN,O
2,NaN,did,NaN,O
3,NaN,evolution,NaN,O
4,NaN,start,NaN,X


In [15]:
data.count()

Sentence #      3651
Word          176691
POS                0
Tag           176691
dtype: int64

In [16]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies = data.Tag.value_counts()
frequencies/len(data)

Number of tags: 2


X    0.538958
O    0.461042
Name: Tag, dtype: float64

In [17]:
tags = {}
for tag, count in zip(frequencies.index, frequencies):
    if tag != "O":
        if tag[2:5] not in tags.keys():
            tags[tag[2:5]] = count
        else:
            tags[tag[2:5]] += count
    continue

In [18]:
labels_to_ids = {k: v for v, k in enumerate(data.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(data.Tag.unique())}
labels_to_ids

{'X': 0, 'O': 1}

In [19]:
# pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
data = data.fillna(method='ffill')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 15,alright,NaN,X
1,Sentence: 15,how,NaN,O
2,Sentence: 15,did,NaN,O
3,Sentence: 15,evolution,NaN,O
4,Sentence: 15,start,NaN,X


In [20]:
# let's create a new column called "sentence" which groups the words by sentence 
data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence 
data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.head()

,Sentence #,Word,POS,Tag,sentence,word_labels
0,Sentence: 15,alright,NaN,X,alright how did evolution start ?[SEP]See above,"X,O,O,O,X,X,O"
1,Sentence: 15,how,NaN,O,alright how did evolution start ?[SEP]See above,"X,O,O,O,X,X,O"
2,Sentence: 15,did,NaN,O,alright how did evolution start ?[SEP]See above,"X,O,O,O,X,X,O"
3,Sentence: 15,evolution,NaN,O,alright how did evolution start ?[SEP]See above,"X,O,O,O,X,X,O"
4,Sentence: 15,start,NaN,X,alright how did evolution start ?[SEP]See above,"X,O,O,O,X,X,O"


In [21]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,alright how did evolution start ?[SEP]See above,"X,O,O,O,X,X,O"
1,Do you not even get statutory sick pay ?[SEP]Y...,"X,X,O,O,O,O,O,O,X,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,Do you not even get statutory sick pay ?[SEP]Y...,"X,O,O,O,O,O,X,X,X,O,O,O,O,O,O,O,O,O,X,X,X,X,X,..."
3,I know of no one that is pro life believes the...,"O,O,X,X,X,X,X,X,X,X,X,O,O,O,O,O,O,O,O,X,X,X,X,..."
4,I know of no one that is pro life believes the...,"X,X,X,X,X,X,X,O,O,O,O,O,O,O,O,O,O,O,O,O,X,X,X,..."


In [22]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [23]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.sentence[index].strip().split()  
        word_labels = self.data.word_labels[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_pretokenized=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            if idx >= len(encoded_labels) or i >= len(labels):
              print(len(encoded_labels), idx, len(labels), i)
            else:
              encoded_labels[idx] = labels[i]
              i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [24]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)

train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (3603, 2)
TRAIN Dataset: (2882, 2)
TEST Dataset: (721, 2)


In [25]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [26]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [27]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.6415, device='cuda:0', grad_fn=<NllLossBackward0>)

In [28]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 2])

In [29]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [30]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.7051356434822083
Training loss per 100 training steps: 0.687646754307322
Training loss per 100 training steps: 0.6763083062954803
Training loss per 100 training steps: 0.6706264529909406
Training loss per 100 training steps: 0.6638780442853818
Training loss per 100 training steps: 0.6609353796688621
Training loss per 100 training steps: 0.6568964309184603
Training loss per 100 training steps: 0.6538492466685776
Training loss epoch: 0.6527926093496992
Training accuracy epoch: 0.6139799920639486
Training epoch: 2
Training loss per 100 training steps: 0.5258387327194214
Training loss per 100 training steps: 0.6028906782664875
Training loss per 100 training steps: 0.6006029851697571
Training loss per 100 training steps: 0.6027319304570803
Training loss per 100 training steps: 0.6023891970255131
Training loss per 100 training steps: 0.6035881278043735
Training loss per 100 training steps: 0.6019470787385538
Training loss per 100 trai

In [31]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [32]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.5706483125686646
Validation loss per 100 evaluation steps: 0.6820152360613984
Validation loss per 100 evaluation steps: 0.6642788338720502
Validation loss per 100 evaluation steps: 0.6735831944154347
Validation Loss: 0.684000439797412
Validation Accuracy: 0.6586249653554432


In [33]:
import os

directory = f"{WORKDIR}/model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)